In [ ]:
import sqlite3
import random
from datetime import datetime, timedelta

# SQLite 데이터베이스 연결
conn = sqlite3.connect('project_management.db')
cursor = conn.cursor()

# 사용자 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY,
    username TEXT UNIQUE,
    email TEXT UNIQUE,
    role TEXT,
    department TEXT
)
''')

# 프로젝트 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS projects (
    project_id INTEGER PRIMARY KEY,
    name TEXT,
    description TEXT,
    start_date DATE,
    end_date DATE,
    status TEXT,
    manager_id INTEGER,
    budget REAL,
    FOREIGN KEY (manager_id) REFERENCES users (user_id)
)
''')

# 작업 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS tasks (
    task_id INTEGER PRIMARY KEY,
    project_id INTEGER,
    assigned_to INTEGER,
    title TEXT,
    description TEXT,
    status TEXT,
    priority TEXT,
    created_date DATE,
    due_date DATE,
    estimated_hours REAL,
    actual_hours REAL,
    FOREIGN KEY (project_id) REFERENCES projects (project_id),
    FOREIGN KEY (assigned_to) REFERENCES users (user_id)
)
''')

# 댓글 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS comments (
    comment_id INTEGER PRIMARY KEY,
    task_id INTEGER,
    user_id INTEGER,
    content TEXT,
    created_date DATE,
    FOREIGN KEY (task_id) REFERENCES tasks (task_id),
    FOREIGN KEY (user_id) REFERENCES users (user_id)
)
''')

# 더미 데이터 생성 함수
def generate_dummy_data():
    # 사용자 데이터 생성
    roles = ["Project Manager", "Developer", "Designer", "Tester", "Business Analyst"]
    departments = ["IT", "Marketing", "Sales", "HR", "Finance"]
    users = []
    for i in range(50):  # 50명의 사용자
        username = f"user_{i+1}"
        email = f"user{i+1}@example.com"
        role = random.choice(roles)
        department = random.choice(departments)
        users.append((username, email, role, department))
    cursor.executemany("INSERT INTO users (username, email, role, department) VALUES (?, ?, ?, ?)", users)

    # 프로젝트 데이터 생성
    projects = []
    for i in range(20):  # 20개의 프로젝트
        name = f"프로젝트 {i+1}"
        description = f"이것은 프로젝트 {i+1}의 설명입니다."
        start_date = datetime.now() - timedelta(days=random.randint(1, 365))
        end_date = start_date + timedelta(days=random.randint(30, 365))
        status = random.choice(["계획", "진행 중", "지연", "완료", "중단"])
        manager_id = random.randint(1, 50)
        budget = random.randint(10000, 1000000)
        projects.append((name, description, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'), status, manager_id, budget))
    cursor.executemany("INSERT INTO projects (name, description, start_date, end_date, status, manager_id, budget) VALUES (?, ?, ?, ?, ?, ?, ?)", projects)

    # 작업 데이터 생성
    tasks = []
    statuses = ["할 일", "진행 중", "검토 중", "완료"]
    priorities = ["낮음", "중간", "높음", "긴급"]
    for project_id in range(1, 21):
        for _ in range(random.randint(10, 30)):  # 각 프로젝트당 10-30개의 작업
            assigned_to = random.randint(1, 50)
            title = f"작업 {random.randint(1, 1000)}"
            description = f"이것은 프로젝트 {project_id}의 작업입니다."
            status = random.choice(statuses)
            priority = random.choice(priorities)
            created_date = datetime.now() - timedelta(days=random.randint(1, 180))
            due_date = created_date + timedelta(days=random.randint(7, 90))
            estimated_hours = random.randint(1, 100)
            actual_hours = estimated_hours + random.randint(-20, 20) if random.random() > 0.5 else 0
            tasks.append((project_id, assigned_to, title, description, status, priority, created_date.strftime('%Y-%m-%d'), due_date.strftime('%Y-%m-%d'), estimated_hours, actual_hours))
    
    cursor.executemany("INSERT INTO tasks (project_id, assigned_to, title, description, status, priority, created_date, due_date, estimated_hours, actual_hours) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", tasks)

    # 댓글 데이터 생성
    comments = []
    for task_id in range(1, len(tasks) + 1):
        for _ in range(random.randint(0, 10)):  # 각 작업당 0-10개의 댓글
            user_id = random.randint(1, 50)
            content = f"이것은 작업 {task_id}에 대한 댓글입니다. {random.choice(['좋은 진행상황입니다!', '추가 정보가 필요합니다.', '마감일을 연장해야 할 것 같습니다.', '이 부분을 다시 검토해 주세요.'])}"
            created_date = datetime.now() - timedelta(days=random.randint(1, 60))
            comments.append((task_id, user_id, content, created_date.strftime('%Y-%m-%d')))
    
    cursor.executemany("INSERT INTO comments (task_id, user_id, content, created_date) VALUES (?, ?, ?, ?)", comments)

# 더미 데이터 생성 실행
generate_dummy_data()

# 변경사항 저장 및 연결 종료
conn.commit()
conn.close()

print("확장된 프로젝트 관리 SQLite 데이터베이스가 성공적으로 생성되었습니다.")